<div >
<img src = "../Machine Learning Aplicado al Marketing-4875-x-834.jpg" />
</div>

# Filtrado Colaborativo Basado en Usuarios.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ignaciomsarmiento/RecomSystemsLectures/blob/main/06_Rec_Usuarios/L06_Colab.ipynb)

Este *cuaderno* trata sobre filtrado colaborativo basado en usuarios. El objetivo del *cuaderno* es que usted obtenga una visión general del problema predictivo de los sistemas de recomendación que utilizan filtrado colaborativo basado en usuario, aprenda distintos algoritmos que lo implementan, y que sea capaz de reconocer sus características, funcionamiento, y  desarrollarlos en `Python`.

# Sistemas de Recomendación

## ¿Qué son los sistemas de recomendaciones?


Las preferencias de los individuos suelen seguir patrones que los sistemas de recomendación pueden aprovechar, por ejemplo,

- Si te interesó:  <div style="max-width:200px">
<img src = "figs/iron_man.jpg" />
</div>



- También te puede interesar: <div style="max-width:200px">
<img src = "figs/thor.jpg" />
</div> 




- Los sistemas de recomendación entonces encuentran patrones que son utilizados para predecir qué otros productos podrían gustarnos y generar sugerencias, de forma tal que  los usuarios encuentren contenido atractivo en un gran corpus. 

- Estos sistemas son muy exitosos, por ejemplo según un estudio del 2013 de [McKinsey](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers), el 35% de los artículos comprados en Amazon surgen de estos sistemas de recomendación, y por lo tanto vale la pena estudiarlos cuidadosamente.



## Introducción: el problema de predicción 

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con $m$ usuarios y $n$ ítems. En esta matriz cada fila representa un usuario y cada columna un ítem. El valor de la celda denota el rating que le dió cada usuario al ítem. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al ítem $j$. Un ejemplo de esta matriz sería la siguiente:

<div  style="max-width: 40%;">
<img src = "figs/fig1.png"/>
</div>

Esta matriz tiene 7 usuarios ($m=7$) y 6 ítems ($n=6$). El usuario 1 le otorgó al ítem 1 un rating de 4, por lo tanto $r_{11}=4$. Sin embargo, no todos los usuarios calificaron/utilizaron todos los ítems y por lo tanto no todos tienen ranking. En el caso del usuario 1 este no utilizó los ítems 3, 4, y 6; por lo tanto aparecen con un signo de interrogación. Estos datos están faltando en la matriz.


Consideremos otro ejemplo más concreto. Imaginemos que somos una compañía como Netflix y tenemos un repositorio de 20.000 películas y 5.000 usuarios. Tenemos además un sistema que registra la calificación que cada usuario le otorga a una película en particular. Es decir, tenemos una  matriz de tamaño 5,000 × 20,000. Sin embargo, es muy probable que los usuarios sólo habrán visto sólo una fracción de las películas (difícilmente todos vieron 20.000 películas!). Por lo tanto, la matriz será poco densa, la mayoría de las entradas en la matriz estarán vacías.


El problema de predicción, tiene por objetivo predecir estos valores faltantes utilizando toda la información que tenemos a disposición: las calificaciones registradas, datos sobre películas, datos sobre usuarios, etc. Si el sistema es capaz de predecir con precisión los valores que faltan, podrá dar excelentes recomendaciones. Por ejemplo, si el usuario $i$ no ha utilizado el ítem $j$, pero nuestro sistema predice una calificación muy alta, es entonces muy probable que le guste $j$ si lo descubre a través del sistema de recomendación.

## Filtrado Colaborativo Basado en Usuarios.

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones. 

 <div style="max-width:400px">
<img src = "figs/Colab.png" />
</div> 



### Ejemplo

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con 5 usuarios y 5 productos, en este caso restaurantes. 

El valor de la celda denota el rating que le dió cada usuario al restaurante. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al restaurante $j$. 
 

In [ ]:
#cargamos librerias
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/Ratings.csv')
ratings.head()

In [ ]:
# pivotamos la tabla para tener una matriz de usuarios y restaurantes
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='restaurant_id')
r_matrix.head()

In [ ]:
# creamos una matriz de usuarios y restaurantes donde remplazamos los valores nulos por 0
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

### Filtrado colaborativo sencillo (simple): medias, y medias ponderadas.


#### Medias 

- Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario 

In [ ]:
 puntuac_promedio = ratings.groupby('restaurant_id').mean()
 pd.DataFrame(puntuac_promedio.iloc[:,1])

#### Medias ponderadas

- Podemos hacer algo más sofisticado usando recomendaciones de usuarios similares?


- Podemos hacer una media ponderada


$$
r_{ur}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'r}}{\sum_{u',u'\neq u}sim(u,u')}
$$



- Es decir la predicción del rating del usuario $u$ para el restaurante $r$, $r_{ur}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a este restaurante, 


- Ponderado por cuán similares son los usuarios $u'$ a $u$. 


- Como medimos similitud?

    - Existen múltiples medidas de distancia que se utilizan para medir la similitud. 

    - La distancia de coseno, que suele ser la más utilizada en los sistemas de recomendación.

    - Matemáticamente

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$


Es decir, es el cociente del producto punto, dividido por las normas de los vectores.

<center>
<img src = "figs/dist_cos.png" alt = "coseno" style = "width: 300px;"/>
</center>


- Si el ángulo es 0  de grados, entonces los vectores se solapan, y el coseno es igual a 1 
- Si el ángulo es 90 de grados, los vectores forman un angulo recto, y el coseno es igual a 0.
- Si el ángulo es 180 de grados, los vectores estan en sentido opuesto, y  el coseno es igual a -1.



In [ ]:
import math

math.cos(math.radians(0))

**¿Qué medida de similitud elegir?** 

Notemos que, si dos vectores, $x$ e $y$, tienen media cero, el coseno del angulo entre ambos coincidirá con el coeficiente de correlación de Pearson.

La distancia del coseno suele ser una medida de similitud ampliamente utilizada en los sistemas de recomendación por varias razones:

   - *Independencia de magnitud*: La distancia del coseno mide la similitud entre dos vectores basándose en el ángulo entre ellos, en lugar de depender de las magnitudes de los vectores. Esto es útil en los sistemas de recomendación, donde las magnitudes de las características (por ejemplo, la calificación de un usuario para un artículo) pueden variar significativamente entre diferentes usuarios o elementos. Al ser independiente de la magnitud, el coseno permite capturar patrones de similitud más robustos.

   - *Espacios de alta dimensión*: En los sistemas de recomendación, las características que representan usuarios o elementos a menudo se representan como vectores en espacios de alta dimensión. En estos espacios, la noción de distancia euclidiana puede volverse menos significativa debido al fenómeno conocido como "maldición de la dimensionalidad". La distancia del coseno puede ser más adecuada en estos casos porque se centra en la orientación relativa de los vectores, lo que permite medir la similitud de manera más efectiva.

   - *Eficiencia computacional*: Calcular la distancia del coseno es computacionalmente eficiente en comparación con otras métricas de similitud más complejas, como la distancia de Mahalanobis o la distancia euclidiana en espacios de alta dimensión. Esto es importante en sistemas de recomendación que manejan grandes conjuntos de datos y deben realizar cálculos rápidos para generar recomendaciones en tiempo real.

   - *Contexto de recomendación*: En muchos sistemas de recomendación, las preferencias de los usuarios pueden estar dispersas, lo que significa que solo han interactuado con una pequeña parte del conjunto total de elementos. La distancia del coseno tiende a funcionar bien en estos escenarios, ya que ignora las diferencias entre características no observadas, lo que puede conducir a una mejor generalización y recomendaciones más precisas.

A pesar de estas ventajas, es importante destacar que no existe un enfoque único para todos los sistemas de recomendación. La elección de la medida de similitud depende en gran medida del contexto y la naturaleza de los datos en un sistema específico. Por ejemplo,

 - En caso que las magnitudes sean importantes, la distancia euclidiana es una medida apropiada para utilizar. 
 - Si la correlación es lo que importa, entonces la correlación de Pearson o el coseno son más apropiadas. 
 - Si la popularidad del ítem bajo estudio es importante, entonces el producto escalar será una medida adecuada. Puesto que si los ítems aparecen con mucha frecuencia (por ejemplo, videos populares de YouTube) estos tienden a normas grandes y el producto escalar capturará mejor esta información. Sin embargo, si no somos cuidadosos, aquellos artículos populares terminarán siendo los más recomendados. En casos como este, se puede definir una medida que "regularice" las normas, como por ejemplo: $similitud(x,y)=|x|^\alpha  |y|^\alpha cos(x,y)$ para algún $\alpha\in(0,1)$.
 

##### Creamos una función recomendadora usando esto

In [ ]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz de similitud en un DataFrame
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim


In [ ]:
?cosine_similarity

In [ ]:
# Definimos una función para calcular el promedio ponderado de los ratings
def cf_user_wmean(user_id, restaurant_id):
    
    # Primero verificamos si el restaurante está en la matriz
    if restaurant_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[restaurant_id]
        
         # Obtenemos los índices de los restaurantes sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [ ]:
cf_user_wmean(1,1)

In [ ]:
sim_scores = cosine_sim[1]
sim_scores

In [ ]:
m_ratings =r_matrix[1]
m_ratings

In [ ]:
idx = m_ratings[m_ratings.isnull()].index
idx

Falta entonces 

$$
r_{um}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'm}}{\sum_{u',u'\neq u}sim(u,u')}
$$


In [ ]:
sim_scores1 = sim_scores.drop(idx)

m_ratings = m_ratings.dropna()
        
# Calculamos la media ponderada
wmean_rating = np.dot(sim_scores1, m_ratings)/ sim_scores1.sum()
wmean_rating

## Un ejemplo más realista y complicado


Para ilustrar cómo construir e implementar este tipo de sistema de recomendación utilizaremos nuevamente los datos de  [MovieLens](https://grouplens.org/datasets/movielens/latest/) provista abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Esta versión de los datos contiene varias bases con información de los usuarios, de las películas, y de los ratings. Tendremos que combinar estas bases para obtener una matriz similar a la ilustrada en la gráfica anterior. Carguemos entonces las librerías y los datos de usuarios:

In [ ]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
# Cargamos los datos de los usuarios
u_cols = ['user_id', 'edad', 'genero', 'ocupacion', 'codigo_postal']

users = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L04_Usuarios/data/u.user', sep='|', names=u_cols,
 encoding='latin-1')
users.head()

Podemos ver que esta base contiene un identificador de usuario, su edad, género, ocupación, y el código postal donde viven. Luego cargamos la base de películas:

In [ ]:
# Cargamos los datos de las películas
i_cols = ['movie_id','titulo', 'fecha_estreno', 'fecha_estreno_video', 'URL_IMDb', 'desconocido', 'Accion', 'Aventura',
  'Animacion', 'Infantil', 'Comedia', 'Crimen', 'Documental', 'Drama', 'Fantasia',
  'Cine-Noir', 'Horror', 'Musical', 'Misterio', 'Romance', 'Ciencia_ficcipn', 'Thriller', 'Guerra', 'Western']

movies = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head()

Esta base contiene un identificador de película, el título, cuándo fue estrenada en el cine, cuándo fue estrenada en video, la dirección web a IMDb, y el género al que pertenece, incluyendo una columna que marca como desconocido si el género no pertenece a ninguna de las restantes. Notemos además que una película puede pertenecer a varios géneros, por ejemplo: Toy Story pertenece  al genero Animación e Infantil. Pero por ahora nos importa sólo el identificador y el título de la película, por lo que nos quedamos con estas columnas:

In [ ]:
## completar, ayuda columnas de interes 'movie_id', 'titulo'
movies = movies[['movie_id', 'titulo']]
movies.head()

La última base que necesitamos cargar es la de ratings:

In [ ]:
# Cargamos los datos de los ratings 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

ratings.head()

Esta base tiene información sobre qué `rating` el usuario (`user_id`) le otorgó a cada película (`movie_id`), e información de cuándo fue que hizo tal calificación (`timestamp`). Esta última columna no la utilizaremos por lo que la excluiremos de la base.

In [ ]:
# Quitamos la columna timestamp 
# llenar
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

Con estos elementos podemos construir la matriz que vincule usuarios, películas y ratings. Para ello usamos la función `pivot_table`.

In [ ]:
## Completar ayuda usar pivot_table siendo valores rating y las columnas movie_id
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

In [ ]:
## completar  saber la dimen con comando shape
r_matrix.shape

Esta matriz entonces vincula los usuarios con las películas. Las filas denotan los 943 usuarios y las columnas las 1682 películas en la base. El usuario 1 le otorgó a la película 1 un rating de 5, por lo tanto $r_{11}=5$. Al igual que en el ejemplo anterior, no todos los usuarios calificaron/vieron todas las películas y por lo tanto no todas tienen ranking. En el caso del usuario 1 este no calificó las películas 1673 a 1682 y por lo tanto aparecen con `NaN`; estos datos están faltando en la matriz.

La tarea es entonces buscar estrategias para completar estos datos faltantes.

### Medias

Comencemos por la estrategia quizás más simple e intuitiva para completar las celdas faltantes. Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario que ranquearon esta película. No hacemos distinción entre los usuarios, y el rating de cada uno recibirá el mismo peso. Habrá casos donde ninguna de las películas ha sido ranqueada, en tales situaciones le pondremos un rating default de 3. 

Para evaluar además la performance de esta estrategia, dividiremos la base original en bases de entrenamiento y prueba.

In [ ]:
# Importamos la función train_test_split 
from sklearn.model_selection import train_test_split
   
#Asignamos `X` como la base original de ratings e `y` el usuario 
   
X = ratings.copy()
y = ratings['user_id']

# Partimos la base en entrenamiento y prueba estratificando por usuario
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

y definiremos como métrica el error cuadrático medio

In [ ]:
# Importamos la función mean_squared_error
from sklearn.metrics import mean_squared_error

# Creamos una funcion que calcula la raíz del error cuadrático medio (RMSE)
def rmse(y_true, y_pred):
       return np.sqrt(mean_squared_error(y_true, y_pred))


Con la base lista podemos construir nuestra función recomendadora:

In [ ]:
## Construir funcion de media
## llenar
def cf_user_mean(user_id, movie_id):
    
    # Primero verificamos si la película está en la matriz
    if movie_id in r_matrix:
        # Si esta calculamos la media de los ratings
        mean_rating = r_matrix[movie_id].mean()
    
    else:
        # Si no lo está, asignamos el valor de 3 (los invito a que prueben con otros valores)
        mean_rating = 3.0
    
    return mean_rating

Podemos entonces evaluar el RMSE que obtendremos de la base de prueba de haber entrenado la función recomendadora en la base de entrenamiento:

In [ ]:
# crear funcion Score modelo
def score(cf_model):
    # Construimos una lista con las tuplas usuario-película en la base de entrenamiento
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    # Predecimos el rating para cada tupla usuario-película
    y_pred = np.array([cf_model(user, movie) for (user, movie) in
   id_pairs])
    # Extraemos los ratings que dieron los usuarios en la base de prueba
    y_true = np.array(X_test['rating'])
    # Retornamos el RMSE
    return rmse(y_true, y_pred)

Aplicamos esta función y obtenemos el RMSE de este modelo

In [ ]:
## aplicar funcion
score(cf_user_mean)

Podemos entonces utlizar este resultado como una base para la comparación a los modelos subsiguientes.

### Medias ponderadas


In [ ]:
# Rellenamos los faltantes con 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [ ]:
r_matrix_dummy

Con esto estamos en condiciones de usar la función `cosine_similarity` que va a calcular la similitud de coseno.

In [ ]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz resultante en un dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head(10)

Tenemos entonces como resultado una matriz que muestra la similitud de coseno entre los diferentes usuarios. La matriz diagonal que muestra $1$ nos dice que la similitud entre el usuario 1 y sí mismo es 1. Note que el 0.16 dice que la similitud entre 1 y 2 es de 0.16. En este caso, dado que estamos evaluando ratings positivos, esta matriz retornará entradas positivas.

Con esta matriz calculada podemos entonces utilizarla para construir nuestra función recomendadora basada en medias ponderadas.

In [ ]:
#Completar
#Crear función recomendadora basada en medias con input user_id y movie_id
def cf_user_wmean(user_id, movie_id):
    
    # Primero verificamos si la película esta en la matriz
    if movie_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[movie_id]
        
         # Obtenemos los índicies de las películas sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [ ]:
cosine_sim[1]

In [ ]:
sim_scores = cosine_sim[1]
r_matrix[1]
m_ratings = r_matrix[1]
idx = m_ratings[m_ratings.isnull()].index
        
# Nos quedamos con similitudes y ratings completos
sim_scores = sim_scores.drop(idx)
m_ratings = m_ratings.dropna()
m_ratings

Estamos en condiciones entonces de evaluar este modelo con la función antes creada:

In [ ]:
score(cf_user_wmean)

Notemos que esta simple "sofisticación" mejora el desempeño predictivo.

### Datos demográficos de los usuarios

El modelo anterior entonces incorporó  similitud entre las calificaciones de los usuarios para mejorar las recomendaciones. ¿Qué sucede entonces si utilizamos variables demográficas? La intuición es que individuos de similares géneros, profesiones, edades, etc., tendrán preferencias más similares que individuos con características demográficas diferentes.

En este caso lo que haremos es restringir a los ratings de individuos con las mismas características demográficas, en jerga estadística, utilizaremos un modelo de medias condicionales.

Para ello, primero unimos los datos originales sobre usuarios con nuestra base de entrenamiento.

In [ ]:
merged_df = pd.merge(X_train, users)

In [ ]:
merged_df.head()

Con esta base que tiene ahora variables demográficas podemos intentar mejorar nuestro recomendador. Ilustremos entonces una función recomendadora basada en género y ocupación. Para ello, primero calculamos el rating promedio por género y ocupación. En `pandas` esto es relativamente sencillo utilizando la función `groupby`.

In [ ]:
users = users.set_index('user_id')

In [ ]:
# Calculamos el rating promedio basado en género y ocupación
# Completar
gen_occ_mean = merged_df[['genero', 'rating', 'movie_id', 'ocupacion']].pivot_table(
    values='rating', index='movie_id', columns=['ocupacion', 'genero'], aggfunc='mean')

gen_occ_mean.head()

Tenemos entonces el promedio de rating por género y ocupación. Así, los administradores femeninos ranquearon la película 1 con un promedio de 3.93 y los masculinos con 3.75. Mientras que los artistas no ranquearon la película 2. Con esta información generamos una función recomendadora que incorpore esta información y en los casos donde no tenemos información utilizaremos el rating 3 como default.

In [ ]:
#Completar función recomendadora que incorpore la información anterior
def cf_gen_occ(user_id, movie_id):
    
    # Verificamos si la película existe en gen_occ_mean
    if movie_id in gen_occ_mean.index:
        
        #Identificamos el usuario
        user = users.loc[user_id]
        
        #Identificamos el género y la ocupación
        gender = user['genero']
        occ = user['ocupacion']
        
        # Verificamos si la ocupación calificó la película
        if occ in gen_occ_mean.loc[movie_id]:
            
            # Verificamos si el género calificó la película
            if gender in gen_occ_mean.loc[movie_id][occ]:
                
                # Obtenemos la calificación
                rating = gen_occ_mean.loc[movie_id][occ][gender]
                
                # Default de 3.0 si no tiene calificación
                if np.isnan(rating):
                    rating = 3.0
                
                return rating
            
    # Default de 3.0 si no tiene calificación
    return 3.0

Evaluamos entonces esta función recomendadora:

In [ ]:
score(cf_gen_occ)

Vemos que esta función es la que peor funciona de las antes evaluadas. La función que incorporaba el coseno como medida de similitud lograba mejores recomendaciones. En la sección siguiente nos centraremos en entender mejor las medidas de similitud y cómo estas nos permiten mejorar los recomendadores.

## Filtrado colaborativo basado en *embeddings*

Como vimos anteriormente parte del problema reside en encontrar productos similares para recomendar. Una forma de lograrlo es utilizando *embeddings*. 

Los *embeddings* (también conocidos como incrustaciones en español) son representaciones numéricas de alta dimensionalidad que se utilizan para codificar conceptos, objetos o entidades en un espacio vectorial. Estas representaciones son aprendidas automáticamente a partir de datos y se caracterizan por capturar características y relaciones entre los elementos. Los embeddings son ampliamente utilizados en diversas áreas de la inteligencia artificial, como el procesamiento del lenguaje natural (PLN), la visión por computadora y los sistemas de recomendación.

Un *embedding* o incrustación implica entonces un mapeo de un conjunto discreto (el conjunto de elementos a recomendar) a un espacio vectorial de menor dimensión llamado *embedding space* o *espacio de incrustación*. Este  *embedding space* es de dimensión considerablemente menor al tamaño del espacio original y captura alguna estructura latente de los elementos. 

Por ejemplo, supongamos que tenemos una lista de números: [1, 2, 3, 4]. Si queremos representar estos números en un espacio de embedding de 1 dimensión, podríamos usar la siguiente codificación:

    - 1 -> 2.5
    - 2 -> 4.0
    - 3 -> 5.5
    - 4 -> 7.0

En este caso, cada número se ha mapeado a un solo valor en *embedding space* de una dimensión.  Es importante tener en cuenta que, en la práctica, los espacios de embedding suelen tener más dimensiones para capturar mejor la complejidad y las relaciones entre los elementos.


### Ejemplo: Embeddings para recomendar películas 

Estudiemos un ejemplo en el que queremos generar recomendaciones de películas a partir de *embeddings*. Supondremos que nuestra plataforma de streaming: **"SlowFlow"** tiene 4 usuarios y 5 películas: [Batman: El caballero de la noche asciende](https://es.wikipedia.org/wiki/The_Dark_Knight_Rises), [Harry Potter y la Piedra Filosofal](https://es.wikipedia.org/wiki/Harry_Potter_y_la_piedra_filosofal), [Shrek](https://es.wikipedia.org/wiki/Shrek), [Las trillizas de Belleville](https://es.wikipedia.org/wiki/Les_Triplettes_de_Belleville) y [Memento](https://es.wikipedia.org/wiki/Memento). 


|                **Película**               | **Clasificación** |                                                                                                  **Descripción**                                                                                                  |
|:-----------------------------------------:|:-----------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Batman: El caballero de la noche asciende |       PG-13       |                              Batman se esfuerza por salvar a Gotham City de la aniquilación nuclear en esta secuela del Caballero de la Noche ambientada en el universo de DC Comics.                             |
|     Harry Potter y la Piedra Filosofal    |         PG        |                       Un niño huérfano descubre que es un mago y se matricula en el Colegio Hogwarts de Magia y Hechicería donde libra su primera batalla contra el malvado Lord Voldemort.                       |
|                   Shrek                   |         PG        |                                Un ogro adorable y su compañero burro se embarcan en una misión para rescatar a la princesa Fiona que está encarcelada en su castillo por un dragón.                               |
|        Las trillizas de Belleville        |       PG-13       | Cuando el ciclista profesional Champion es secuestrado durante el Tour de Francia, su abuela y su perro con sobrepeso viajan al extranjero para rescatarlo con la ayuda de un trío de ancianos cantantes de jazz. |
|                  Memento                  |         R         |                                                       Un amnésico busca desesperadamente resolver el asesinato de su esposa tatuándose pistas en su cuerpo.                                                       |


La siguiente matriz que llamaremos $A$, donde $A\in\mathbb{R}^{ m\times n}$, es decir, las filas son los $m$ usuarios y las columnas las $n$ películas, muestra con un $\checkmark$ las películas que cada usuario vió:


<center>
<img src = "figs/colab0.png" alt = "embedding1D" style = "width: 500px;"/>
</center>


El objetivo final del ejemplo será generar recomendaciones de películas a nuestros usuarios que no hayan visto.


## ¿Cómo funcionan los embeddings?

### Embedding en una sola dimensión 

Comencemos representando las películas y los usuarios a un *embedding space*  de una sola dimensión. Por ejemplo, a lo largo del segmento $[-1,1]$ de forma tal que películas e individuos similares estén más cerca.

Uno de las posibles representaciones que podemos hacer es asignar un escalar que describa la película a lo largo de la dimensión: niños $\rightarrow$ adultos. El escalar va a asignar valores negativos si la película es para niños y positivos si es para adultos. Es decir, estamos resumiendo toda la película a una sola dimensión.  La figura a continuación muestra este posible mapeo:

<center>
<img src = "figs/colab1.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

Mapeamos:

- Shrek -> -1
- Las trillizas de Belleville -> -0.8
- Harry Potter -> -0.2
- Batman -> 0.9
- Memento -> 1

Entonces Shrek que es para niños fue mapeada a -1, mientras que  Memento, que es para adultos, a  1. El resto de las películas fueron mapeadas a lo largo de esta dimensión. 

Por otro lado, podemos mapear a los 4 individuos a la dimensión de  preferencias por películas para niños o para adultos. Así por ejemplo, podríamos tener el siguiente mapeo: 

<center>
<img src = "figs/colab2b.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

donde:

1. Amarillo -> -1
2. Azul -> 0
3. Rojo -> 0.1
4. Verde -> 1

Es decir, el individuo amarillo que prefiere películas de niños fue mapeado a 1 películas de niños, mientras que el individuo verde a 1 porque prefiere películas de adultos.

Esta dimensión es útil ya que nos permitiría recomendar películas de niños a aquellos individuos que sólo ven películas de niños, o películas de adultos para aquellos que sólo ven este tipo de películas. Sin embargo hay otras dimensiones que no están siendo capturadas y que potencialmente ayudarían a mejorar las recomendaciones. Continuando con el ejemplo, agreguemos una segunda dimensión.


#### Embeddings en dos dimensiones

Agreguemos ahora otra dimensión: la popularidad de película. Mediremos esta como el grado en que cada película es [cine de culto](https://es.wikipedia.org/wiki/Pel%C3%ADcula_de_culto) o es un éxito de taquilla. En este caso el escalar nuevamente estará entre $[-1,1]$ y tomará -1 si es una película de culto y 1 si es un éxito de taquilla. 

Con esta segunda característica, podemos ubicar las películas en un espacio bidimensional: 

<center>
<img src = "figs/colab4.png" alt = "embedding2D" style = "width: 500px;"/>
</center>


Shrek que fue un éxito de taquilla y una película de niños se ubica en el cuadrante superior izquierdo, mientras que Memento, una película de culto y para adultos esta en el cuadrante inferior derecho. En este caso, cada película se ha mapeado a un vector de 2 dimensiones en el *embedding space*. Las películas con contextos similares tienden a tener embeddings más cercanos entre sí, lo que facilita la identificación de similitudes en el espacio vectorial.

De manera más general, lo que estamos haciendo es reducir la dimensión de las películas a un *embedding space* . En este espacio cada película se describe mediante un conjunto de coordenadas. Por ejemplo, en este espacio, "Shrek" tiene las coordenadas  (-1, 0,95) y "Memento" (1, -0,5). En general, cuando mapeamos a un espacio $d-dimensional$, cada película se representa mediante $d$ números con valores reales, cada uno de los cuales da la coordenada en una dimensión.

De la misma forma, podemos mapear a los individuos en este espacio:

<center>
<img src = "figs/colab5.png" alt = "embedding2D" style = "width: 500px;"/>
</center>


Podemos ver la información de los *embeddings* que obtuvimos en los pasos anteriores en la siguiente matriz:

<center>
<img src = "figs/colab3b.png" alt = "embedding2D" style = "width: 500px;"/>
</center>

Donde las filas representan a los individuos y el mapeo a las distintos dimensiones dependiendo sus preferencias. Las columnas mientras tanto representan a los películas y el mapeo a las distintas dimensiones.

En estos ejemplos, diseñamos a mano los *embeddings* y nombramos estas dimensiones. En la práctica, los *embeddings*  son aprendidas de los datos lo que lleva a que a menudo las dimensiones individuales no tienen un nombre o interpretación clara. 

#### Recomendaciones a partir de Embeddings

En los pasos anteriores construimos dos  *embeddings spaces* uno para las películas y otro para los usuarios. Una ventaja de estos espacios es que nos van a permitir construir recomendaciones. 

Con los *embeddings* de los productos podemos construir una matriz de *embeddings* de elementos $V\in\mathbb{R}^{ n\times d}$, donde la fila $j$ es el *embedding* del elemento $j$, que en nuestro ejemplo estará dada por la matriz de *embeddings* de las películas construidas en el paso anterior

 $$
V=\left(\begin{array}{cc}
0.9 & -0.2\\
-1 & -0.8\\
1 & -1\\
1 & 0.9\\
-0.9 & 1
\end{array}\right)
$$

donde la primer columna es la dimensión *culto-> taquillera* y el segundo *niños -> adultos*. De la misma forma tendremos una matriz de  *embeddings* de usuarios $U\in\mathbb{R}^{ m\times d}$ donde la fila $i$ es el *embedding* del usuario $i$:

$$
U=\left(\begin{array}{cc}
1 & 0.1\\
-1 & 0\\
0.2 & -1\\
0.1 & 1
\end{array}\right)
$$

Con estos *embeddings* vamos a buscar  reconstruir la matriz que contiene la información sobre si los usuarios vieron o no la película, la matriz $A\in\mathbb{R}^{ m\times n}$ 

$$
A=\left(\begin{array}{ccccc}
1 & 0 & 1 & 1 & 0\\
0 & 1 & 0 & 0 & 1\\
1 & 1 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 1\\
\end{array}\right)
$$

donde 1 indica que el usuario vio la película, y 0 que no. Lo que vamos a hacer aproximar $A$ por el producto de los embeddings $UV^T$.

Haciendo la operación en Python

In [ ]:
import numpy as np
U = np.array([[1,0.1], [-1,0], [0.2,-1],[0.1,1]])
VT = np.transpose(np.array([[0.9,-0.2],[-1,-0.8],[1,-1],[1,0.9],[-0.9,1]]))
np.dot(U,VT)

Tenemos entonces una aproximación a la matriz $A$ pero también obtenemos información adicional en las entradas que antes tenían cero. Esta información puede ser utilizada para generar las recomendaciones. Asi por ejemplo, al cuarto usuario primero le recomendaríamos Shrek, mientras que al segundo usuario le recomendaríamos Batman.


<center>
    <img src = "figs/colab7c.png" alt = "embedding2D" style = "width: 700px;"/>
</center>

### Generación de *Embeddings*

Finalmente queda discutir como generar estas matrices de *embeddings*. La sección anterior nos da una idea de como hacerla, dada la matriz observada $A\in\mathbb{R}^{ m\times n}$ con $m$ usuarios y $n$ ítems, queremos encontrar una representación de esta matriz en una dimensión menor y que capture las relaciones subyacentes entre usuarios y películas.


### Factorización de matrices

 La factorización de matrices es un modelo simple que nos permite generar estos *embeddings*.

Dada la matriz $A\in\mathbb{R}^{ m\times n}$ con $m$ usuarios y $n$ ítems, buscaremos que el modelo genere:

- Una matriz de *embeddings* de usuarios $U\in\mathbb{R}^{ m\times d}$ donde la fila $i$ es el *embedding* del usuario $i$.
- Una matriz de *embeddings* de elementos $V\in\mathbb{R}^{ n\times d}$, donde la fila $j$ es el *embedding* del elemento $j$.

Los *embeddings* se generaran de forma tal que el producto $UV^T$ sea una buena representación de la matriz $A$. Volviendo al ejemplo anterior, $U$ estará dado por:

$$
U=\left(\begin{array}{cc}
1 & .1\\
-1 & 0\\
.2 & -1\\
.1 & 1
\end{array}\right)
$$

y $V$ estará dada por 

$$
V=\left(\begin{array}{cc}
.9 & -.2\\
-.8 & -.8\\
1 & -1\\
1 & .9\\
-.9 & 1
\end{array}\right)
$$

De forma tal que el producto aproxime de la matriz que indica que película ha visto:


<center>
<img src = "figs/colab7c.png" alt = "embedding2D" style = "width: 500px;"/>
</center>


En nuestro caso vamos al descomponer la matriz de ratings $A$, que describe las interacciones usuario-películas, nos encontramos con tres matrices de menor dimensión: 

- $U$  la matriz de *embeddings* de usuarios, 
- $V$ la  matriz de *embeddings* de películas, 
- $S$ la matriz diagonal de los valores singulares.

La matriz diagonal de valores singulares ($S$) es fundamental para comprender la estructura latente que se captura durante el proceso de SVD en sistemas de recomendación. Estos valores singulares están ordenados de mayor a menor, lo que nos permite truncar la descomposición para conservar solo los primeros $d$ valores singulares más importantes reduciendo efectivamente la dimensión. Estos $d$ valores singulares determinarán también la dimensión del *embedding space*. 

La estructura latente capturada por los valores singulares se refiere a las relaciones y patrones subyacentes en los datos originales que son descubiertos por la SVD. Cada valor singular en la matriz $S$ representa la importancia de una dimensión específica en la representación de la matriz original. Al estar ordenados de mayor a menor, significa que los primeros valores singulares capturan la mayor cantidad de información relevante, mientras que los valores singulares más pequeños representan información menos relevante o ruido.

Cuando seleccionamos solo los primeros $d$ valores singulares más importantes para reconstruir la matriz original, estamos reduciendo la dimensionalidad de los datos. La dimensionalidad reducida se refiere a que solo estamos considerando las $d$ dimensiones más significativas para representar los usuarios y películas en un espacio de *embeddings* 

Esta reducción de dimensionalidad permite que el modelo capture de manera más eficiente las relaciones y similitudes entre usuarios y películas, ya que las dimensiones menos relevantes o ruidosas se descartan. Esencialmente, los *embeddings* resultantes de SVD se encuentran en un espacio de menor dimensión, pero aún conservan una representación significativa de los patrones de interacción entre usuarios y películas. Es por ello que la selección del número adecuado de valores singulares es un paso crítico en la aplicación de SVD para sistemas de recomendación. Un valor demasiado pequeño puede resultar en una pérdida significativa de información y, por lo tanto, en recomendaciones inexactas. Por otro lado, un valor demasiado grande puede llevar a problemas de sobreajuste y un aumento en la complejidad computacional.

La estructura latente capturada por los valores singulares se puede interpretar como características o conceptos latentes que subyacen a las interacciones entre usuarios y películas. Estas características pueden representar preferencias de los usuarios, intereses específicos o temáticas de los películas. En nuestro ejemplo las características latentes eran si la película era para niños o adultos y su éxito en la taquilla. Pero podríamos tener otras  características latentes que en ocasiones, podemos inspeccionar y asignarle una interpretación. Sin embargo, generalmente esto no será posible ya que estarán capturando una dimensión subyacente, ya que no representa una característica no explícita en los datos sino que se deduce de ellos.

Al capturar estas características latentes, SVD permite una representación más compacta y significativa de los usuarios y las películas, lo que facilita la recomendación eficiente de películas similares a los que el usuario ha mostrado interés previamente.


En resumen, los valores singulares en la SVD nos permiten identificar y representar las características latentes más importantes en los datos originales, lo que facilita la recomendación precisa y eficiente de elementos para los sistemas de recomendación basados en usuarios.


####  Interpretación de Embeddings

##### Ejemplo 1:

In [ ]:
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
pelis = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/pelis.csv')
pelis = pelis.set_index("Usuarios")
pelis

In [ ]:
np.linalg.matrix_rank(pelis.to_numpy())

In [ ]:
U, S, Vt = np.linalg.svd(pelis, full_matrices=False)

In [ ]:
print(U[:,0:2])

In [ ]:
print(Vt[0:2,:])

In [ ]:
n=2
k=2
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal

print(S2)

In [ ]:
#Reconstrucción
l=2
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

##### Ejemplo 2:

In [ ]:
# Cargamos y visualizamos  los datos
pelis2 = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/pelis2.csv')
pelis2 = pelis2.set_index("Usuarios")
pelis2

In [ ]:
np.linalg.matrix_rank(pelis2.to_numpy())

In [ ]:
U, S, Vt = np.linalg.svd(pelis2,full_matrices=False)
print('U, S, Vt ='), U, S, Vt

In [ ]:
n=3
k=3
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal
l=3
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

# Ejemplo 3


<center>
<img src = "figs/colab7c.png" alt = "embedding2D" style = "width: 500px;"/>
</center>

In [ ]:
#Importamos libreria numpy
import numpy as np
#Completar creación de matriz y matriz transpuesta
U = np.array([[1,0.1], [-1, 0], [0.2,-1],[0.1,1]])
VT = np.transpose(np.array([[0.9,-0.2], [-1, -0.8], [1,-1],[1,0.9],[-0.9,1]]))

In [ ]:
U

In [ ]:
VT

In [ ]:
#Hacer producto punto entre ambas matrices
np.dot(U,VT)


Al factorizar estamos buscando una  representación más compacta que aprender la matriz completa. Particularmente, la matriz completa tiene $O(nm)$ entradas, mientras que las matriz de *embeddings*,  $U$ y $V$ tienen $O((n+m)d)$ entradas, donde la dimensión $d$ suele ser mucho más pequeña que $m$ y $n$.

Como resultado, la factorización de matrices encuentra una estructura latente en los datos, asumiendo que las observaciones se encuentran cerca de un subespacio de baja dimensión. En el ejemplo anterior, los valores de $n$, $m$ y $d$ son tan pequeño que la ventaja es insignificante. Sin embargo, en los sistemas de recomendación reales, la factorización de matrices puede ser significativamente más compacta que aprender la matriz completa, como vimos con la reducción de dimensión de las imágenes. 

#### Ejemplo MovieLens 



In [ ]:
## Crear pivot table de ratings con filas de usuarios y columnas los ids de peliculas con valores de ratings
R_df = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
R_df.head()

Tenemos nuevamente un dataframe con los usuarios en las filas y las películas en la columnas. Necesitamos ahora transformarlas a una matriz y centrar en cero utilizando la media de los ratings de cada usuario.

In [ ]:
## Realizar media de rating de una pelicula segun los usuarios
R = R_df.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

En este cuaderno usaremos [SciPy](https://scipy.org/) es que permite elegir los valores singulares como argumento de la función en vez de tener que truncar la matriz posteriormente. Los invito a que prueben utilizando [Numpy](https://numpy.org/).

En este ejemplo, utilizaremos 50 para ilustrar.

In [ ]:
from scipy.sparse.linalg import svds
# Completar SVD usando como parametro 50 valores singulares
U, sigma, Vt = svds(R_demeaned, k = 50)

Para realizar nuestra recomendación tenemos que reconstruir la matriz $A$ con esta aproximación menor. Es decir tenermos que multiplicar $U$, $\Sigma$, and $V^{T}$ para obtener la aproximación de rango $k=50$ de $A$. Puesto que centramos en cero los ratings, necesitamos agregarlos nuevamente.

Haciendo esto tenemos:


In [ ]:
#Recrear la base de recomendaciones
sigma = np.diag(sigma)

db_recomendaciones = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

Notemos que eligimos $k=50$ de forma arbitraria. Si queremos optimizar este recomendador, tendríamos que proceder de forma similar a cómo lo hicimos al inicio de este *cuaderno*, dividir la muestra en entrenamiento y prueba, y buscar el número $k$ que minimize el RMSE. Los invito a que lo hagan por su cuenta y comparen los resultados con los recomendadores que construimos anteriormente.

###### Recomendando películas.

Con la matriz de recomendaciones podemos construir una función que genere recomendaciones a cualquier usuario.

Esta función generará recomendaciones de películas para un usuario específico de forma tal que omita a aquellas que ya ha ranqueado. Estamos asumiendo que si la calificó es porque ya la vió, pero como comparación retornaremos por separado aquellas películas ya calificadas.

Comenzamos poniendo las recomendaciones en un dataframe:

In [ ]:
recomendac_df = pd.DataFrame(db_recomendaciones, columns = r_matrix.columns)
recomendac_df.head()

Con este data frame armamos nuestra función recomendadora:

In [ ]:
#Completar
#Crear función de recomendador
def recomendador(predictions_df, user_id, movies_df, original_ratings_df, num_recommendations=5):
    
    # Obtenemos y ordenamos los usuarios 
    user_row_number = user_id - 1 # user_id inicia en 1, no 0, corregimos el índice
    sorted_user_predictions = recomendac_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Obtenemos los datos de los usuarios y le agremamos los datos de la película
    user_data = original_ratings_df[original_ratings_df.user_id == (user_id)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('El usuario {0} ha calificado {1} películas.'.format(user_id, user_full.shape[0]))
    print('Recomendando las {0} películas que no han sido calificadas.'.format(num_recommendations))
    
    # Generamos la recomendación de las películas que no ha visto todavía
    recomendaciones = (movies_df[~movies_df['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recomendaciones

Notemos que la función toma como argumentos la base que construimos de recomendaciones predichas, el número de usuario, la base de películas y el número de películas que queremos que recomiende.

Notemos que agregamos la base de películas a la función para poder incluir características explícitas de las películas, aunque no las utilizamos en la generación de los *embeddings* y por lo tanto en la recomendación. 

In [ ]:
calificadas, recomendaciones = recomendador(recomendac_df, 837, movies, ratings, 10)

Veamos entonces las recomendaciones generadas. El usuario 837, ha calificado 46 películas, entre las 10 en su tope tiene películas de romance com 'Jane Eyre' e infantiles como 'Willy Wonka and the Chocolate Factory':

In [ ]:
calificadas.head(10)

Veamos las recomendaciones:

In [ ]:
recomendaciones

En estas recomendaciones aparecen películas de romance como "Il Postino" e infantiles como "Toy Story". Estas parecen ser recomendaciones bastante buenas. Notemos además que a pesar de que no incluimos el género como característica en la factorización, esta fue capaz de captar el *embedding* con esta dimensión.

# Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Covington, P., Adams, J., & Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190 

- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022. Disponible en https://developers.google.com/machine-learning/recommendation/overview 

- Google developers. (n.d.). Embeddings: Motivation From Collaborative Filtering. Consultado en Mayo 13, 2022. Disponible en  https://developers.google.com/machine-learning/crash-course/embeddings/motivation-from-collaborative-filtering
